# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [91]:
!cat publications.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [92]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [93]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,url_slug,paper_url,project_url
0,2011-11-14,PolyUCOMP in TAC 2011 Entity Linking and Slot-...,In Proceedings of the Fourth Text Analysis Con...,This paper describes PolyUCOMP team's approach...,tac2011,https://hunterhector.github.io/files/papers/Ji...,NaN
1,2011-07-24,High Performance Clustering for Web Person Nam...,SIGIR2011 Workshop on Entity Orientation Search,We identify a topic drift problem in clusterin...,sigir2011,https://hunterhector.github.io/files/papers/Li...,NaN
2,2012-01-01,Aggregating skip bigrams into key phrase-based...,KONVENS 2012,NaN,konvens2012,https://hunterhector.github.io/files/papers/Xu...,NaN
3,2018-01-13,Graph-Based Decoding for Event Sequencing and ...,7th International Conference on Computational ...,NaN,coling2018,http://arxiv.org/abs/1806.05099,NaN
4,2018-09-03,Automatic Event Salience Identification,EMNLP 2018,NaN,emnlp2018,http://arxiv.org/abs/1809.00647,NaN
5,2017-11-13,Multi-lingual Extraction and Integration of En...,TAC 2017,NaN,tac2017,https://tac.nist.gov/publications/2017/partici...,NaN
6,2017-11-13,"Events Detection, Coreference and Sequencing: ...",TAC 2017,NaN,tac2017,https://tac.nist.gov/publications/2017/present...,NaN
7,2018-03-03,Towards Better Text Understanding and Retrieva...,SIGIR 2018,NaN,sigir2018,https://doi.org/10.1145/3209978.3209982,http://boston.lti.cs.cmu.edu/appendices/SIGIR2...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [94]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [95]:
import os
for row, item in publications.iterrows():    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"

# Too many URL and ugly    
#     if len(str(item.paper_url)) > 5:
#         md += "\npaperurl: '" + item.paper_url + "'"
    
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    paper_url = '#'
    if len(str(item.paper_url)) > 5:
        paper_url = item.paper_url
        
    project_url = '#'
    if len(str(item.project_url)) > 5:
        project_url = item.project_url
        
    md += "\n[Paper](" + paper_url + ")"
    md += " | "
    md += "[Project Page](" + project_url + ")\n"
        
#     md += "\nRecommended citation: " + item.citation
        
    md_filename = os.path.basename(md_filename)
           
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [96]:
!ls ../_publications/

2011-07-24-sigir2011.md    2017-11-13-tac2017.md     2018-09-03-emnlp2018.md
2011-11-14-tac2011.md	   2018-01-13-coling2018.md
2012-01-01-konvens2012.md  2018-03-03-sigir2018.md


In [98]:
!cat ../_publications/2018-03-03-sigir2018.md

---
title: "Towards Better Text Understanding and Retrieval through Kernel Entity Salience Modeling"
collection: publications
permalink: /publication/2018-03-03-sigir2018
date: 2018-03-03
venue: 'SIGIR 2018'
---
[Paper](https://doi.org/10.1145/3209978.3209982)[Project Page](http://boston.lti.cs.cmu.edu/appendices/SIGIR2018-KESM/)
